# Model Entailment 2

This notebook shows how to use entailments on SysML v2 models to look at specific behavior executions on a example car charger.

In [ ]:
import json
import pymbe.api as pm

from pathlib import Path

from typing import Any, Collection, Dict, List, Tuple, Union

from pymbe.model import Model, Element
from pymbe.model_modification import build_from_classifier_pattern, \
                                    new_element_ownership_pattern, \
                                    build_from_binary_relationship_pattern, \
                                    build_unioning_superset_classifier, \
                                    build_from_feature_pattern, \
                                    build_from_binary_connector_pattern, \
                                    apply_covered_feature_pattern

from uuid import uuid4

## Load up SysML Model

Load up a basic model to explore the entailment logic. This version has the derived and implicit elements embedded with it to help with exploring data for the entailment.

In [ ]:
filename = "Concept of Operations Case with Derived"

if not filename.endswith(".json"):
    filename += ".json"

json_file = Path(Path.cwd()).parent.parent / "tests/fixtures" / filename

sysml_model = pm.Model.load_from_post_file(json_file)
sysml_model

In [ ]:
len(sysml_model.elements)

In [ ]:
parts = [ele for ele in sysml_model.elements.values() if ele._metatype in ('PartDefinition', 'PartUsage')]
parts

In [ ]:
actions = [ele for ele in sysml_model.elements.values() if ele._metatype in ('ActionDefinition', 'ActionUsage')]
actions

Gather up memberships in order to find action-action hierarchy to identify the subactions within each major action.

In [ ]:
memberships = [ele for ele in sysml_model.elements.values() if ele._metatype in ('OwningMembership', 'FeatureMembership')]
memberships[0:5]

Isolate the FeatureMemberships that link an ActionDefinition or ActionUsage to children.

In [ ]:
action_action_memberships = [ele for ele in memberships
                             if ele.source[0]._metatype in ('ActionUsage', 'ActionDefinition') and 
                            ele.target[0]._metatype in ('ActionUsage')]
action_action_sources = [ele.source[0] for ele in action_action_memberships]

parent_memberships = [ele for ele in action_action_memberships if ele.source[0] in set(action_action_sources)]
parent_memberships

## Cover semantic library features

In the execution / interpretation examples, new instances of actions are created and rolled into a combined set (type) that is then used as the type for a new feature redefining the original from the library, closing it. 

For example:

```
#atom
behavior MyDry specializes Dry;
     
#atom
assoc MyPaint_Before_Dry_Link specializes HappensBefore {
    end feature redefines earlierOccurrence : MyPaint;
    end feature redefines laterOccurrence : MyDry;
}

behavior MyManufactureStepsPD unions MyPaint, MyDry;
     
#atom
behavior MyManufacture specializes Manufacture {
    feature redefines timeEnclosedOccurrences : MyManufactureStepsPD [2];
    step redefines paint : MyPaint;
    step redefines dry : MyDry [1];
    succession redefines p_before_d : MyPaint_Before_Dry_Link [1]
        first paint then dry;
}
```

Isolate the features of interest in closing the library definitions with a specific result.

In [ ]:
subactions_feature = [ele for ele in actions if ele.declaredName == 'subactions'][0]
subactions_feature

In [ ]:
start_feature = [ele for ele in actions if ele.declaredName == 'start'][0]
start_feature

In [ ]:
done_feature = [ele for ele in actions if ele.declaredName == 'done'][0]
done_feature

Gather up unique sets of the actions that need subactions covered under them.

In [ ]:
action_breakout_dict = {}

for aa_source in set(action_action_sources):
    new_children = []
    for tgt in [ele.target[0] for ele in parent_memberships if ele.source[0] == aa_source]:
        new_children.append(tgt)
    action_breakout_dict.update({aa_source: new_children})
    
action_breakout_dict

Get the base package in the model in order to have a place to put the newly generated actions.

TODO: Move this over to a package specific to the analysis or an analysis case.

In [ ]:
base_package = [ele for ele in sysml_model.ownedElement if ele._metatype == "Namespace"]
base_package[0].throughOwningMembership

In this section, we look at all the subactions of an action, create exactly one specialization for them, bring them together into a set (Classifier), and then redefine the subactions feature in order to type it by that set, closing it. This is an example of closing a feature from the library in order to remove all degrees of freedom from it.

In [ ]:
for action_key in action_breakout_dict.keys():
    new_specials = []
    for subaction in action_breakout_dict[action_key]:
        added_dict = {
            'isIndividual': False,
            'isConjugated': False,
            'isVariation': False,
        }
        action_owning_membership = [ele for ele in memberships if ele.target[0] == subaction]
        
        print(action_owning_membership[0].source[0])
        
        new_special = build_from_classifier_pattern(
            owner=action_key,
            name=f"{subaction.declaredName} as run",
            model=sysml_model,
            specific_fields=added_dict,
            metatype="ActionDefinition"
        )
        
        new_specials.append(new_special)
        
    # create a specialized version of the original action
    
    top_elements = [ele for ele in sysml_model.ownedElement if ele._metatype == "Namespace"][0].throughOwningMembership
    top_package = [ele for ele in top_elements if ele._metatype == "Package"][0]
    
    action_breakout_dict_special = build_from_classifier_pattern(
        owner=top_package,
        name=f"{action_key.declaredName} as run",
        model=sysml_model,
        specific_fields=added_dict,
        metatype="ActionDefinition"
    )
    
    covering_feature = apply_covered_feature_pattern(
        one_member_classifiers=new_specials,
        feature_to_cover=subactions_feature,
        type_to_apply_pattern_on=action_breakout_dict_special,
        model=sysml_model,
        new_types_owner=top_package,
        covering_classifier_prefix = action_key.declaredName + ' ',
        covering_classifier_suffix = ' as run',
        redefining_feature_prefix = '',
        redefining_feature_suffix = ' (Closed)'
    )

In [ ]:
action_breakout_dict_special.throughFeatureMembership[0].throughFeatureTyping[0]

In [ ]:
action_breakout_dict_special.throughFeatureMembership[0].throughFeatureTyping[0].throughUnioning

In [ ]:
action_breakout_dict_special.throughFeatureMembership[0].throughFeatureTyping[0].ownedRelationship

In [ ]:
unionings = [ele for ele in sysml_model.elements.values() if ele._metatype in ('Unioning')]
unionings

The library hierarchy for subactions goes down:

subactions

...actions

...subperformances

......enclosedPerformances

.........performances

.........timeEnclosedPerformances

......suboccurrences

What this means is all subactions are subperformances which are fully enclosed in time without the larger action.

In addition to covering the subactions feature, we can also cover the start and done features.

In [ ]:
all_actions = set([ele.source[0] for ele in parent_memberships]) | set([ele.target[0] for ele in parent_memberships])
all_actions

In [ ]:
top_elements = [ele for ele in sysml_model.ownedElement if ele._metatype == "Namespace"][0].throughOwningMembership
top_package = [ele for ele in top_elements if ele._metatype == "Package"][0]

for action in all_actions:
    #for subaction in action_breakout_dict[action_key]:
    
    solved_action = create_new_classifier(
        owner=top_package,
        name=f"{action.declaredName} as run",
        model=sysml_model,
        added_fields=added_dict,
        metatype="ActionDefinition"
    )
    
    for side in ('startShot', 'endShot'):
    
        added_dict = {
            'isIndividual': False,
            'isConjugated': False,
            'isVariation': False,
        }

        new_special = create_new_classifier(
            owner=top_package,
            name=f"{action.declaredName} {side}",
            model=sysml_model,
            added_fields=added_dict,
            metatype="ActionDefinition"
        )
        
        feature_to_cover = None
        
        if side == 'startShot':
            feature_to_cover = start_feature
        elif side == 'endShot':
            feature_to_cover = done_feature

        cover_library_feature(
            individuals=[new_special],
            feature=feature_to_cover,
            covered_featured_type=solved_action,
            model=sysml_model
        )
        
        print(f"Loaded {new_special} under {solved_action}")

## Develop Timeslices for Car

The actions that have assignment subactions are intended to set values within the car structure at specific times in a given scenario.